In [1]:
# 忠于原版paper的deepfm算法
# 大量参照https://github.com/ChenglongChen/tensorflow-DeepFM
# 他对deep fm进行了一点点优化，虽然数据描述不太明白，但是其数据结构设计得确实好，很精简也非常适合这个算法
# deep fm只能处理one-hot类型的，无法直接处理multi-hot特征
import numpy as np
import tensorflow as tf


In [2]:
"""
参数含义见DeepFMEx
"""
class EmbeddingLayer(tf.keras.layers.Layer):
    def __init__(self,feat_dim,field_num,emb_dim,*args,**kwargs):
        super(EmbeddingLayer,self).__init__(*args,**kwargs)
        self.feat_dim=feat_dim
        self.field_num=field_num
        self.emb_dim=emb_dim
        self.emb_layer=tf.keras.layers.Embedding(input_dim=feat_dim,output_dim=emb_dim)

    def call(self, inputs, **kwargs):

        """

        :param inputs: (feat_indices_batch, feat_value_batch)，分为两部分，你可以把他看做一个手动构造的稀疏矩阵
            例如，如果feat_indices_batch的数据为[[1,5,9],[2,7,8]]；feat_value_batch的数据为[[1,1,2.3],[1,1,0.98]]
            假设第一个样本的特征向量是x，那么x[1]=1, x[5]=1, x[9]=2.3，其余位置取值均为0。
            这样构造是因为每个样本都有field_num个field，每个field的取值只有一种（one-hot或者连续值）
            也就是说每个样本都有field_num个不为0的特征维度。而deep fm算法的嵌入方法是对每一个field嵌入，不管是不是连续值都要嵌入，然后再乘以特征取值
            例如x[9]=2.3，那就要从emb_table里找到第9个emb_vector，然后乘以2.3
        :param kwargs:
        :return:
        """

        # feat_indices_batch: [batch_size, field_num]
        # feat_value_batch: [batch_size, field_num]
        feat_indices_batch,feat_value_batch=inputs
        # 两者形状要相同，并且二者的第二个轴取值维度都是field_num个
        assert feat_indices_batch.shape==feat_value_batch.shape
        assert feat_indices_batch.shape[1:]==[self.field_num]

        emb_vectors=self.emb_layer(feat_indices_batch) # [batch_size, field_num, emb_dim]
        feat_value_batch = tf.expand_dims(feat_value_batch,axis=-1) # [batch_size, field_num, 1]

        # broadcast性质 feat_value_batch会被看做[batch_size, field_num, emb_dim]
        emb_vectors = tf.multiply(emb_vectors,feat_value_batch) # [batch_size, field_num, emb_dim]
        return emb_vectors


feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
print("feat_indices_arr")
print(feat_indices_arr) #[10,3]

feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)
print("\nfeat_vals_arr")
print(feat_vals_arr) # [10,3]

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)


print("\nemb vectors")
emb_layer=EmbeddingLayer(feat_dim=10,field_num=3,emb_dim=4)
emb_vectors=emb_layer(input_batch)
print(emb_vectors)


feat_indices_arr
[[5. 9. 6.]
 [7. 5. 2.]
 [3. 0. 9.]
 [2. 6. 3.]
 [4. 9. 0.]
 [4. 1. 6.]
 [5. 7. 6.]
 [8. 5. 9.]
 [3. 5. 7.]]

feat_vals_arr
[[1.         1.         0.09902062]
 [1.         1.         0.09132659]
 [1.         1.         0.13841131]
 [1.         1.         0.83238316]
 [1.         1.         0.61221683]
 [1.         1.         0.19916216]
 [1.         1.         0.31895494]
 [1.         1.         0.98576325]
 [1.         1.         0.3094537 ]]

emb vectors
tf.Tensor(
[[[ 0.02018002 -0.00491751 -0.03456549 -0.03287257]
  [-0.00180503 -0.01379727  0.03490532 -0.04411305]
  [-0.00482481  0.00070595  0.00403068 -0.00438183]]

 [[-0.02833303 -0.04482388 -0.00266     0.03493133]
  [ 0.02018002 -0.00491751 -0.03456549 -0.03287257]
  [-0.00118857  0.00307725  0.00431112 -0.00169061]]

 [[ 0.0432649  -0.00515907 -0.03374316  0.0488158 ]
  [ 0.03074877 -0.04617167  0.00308106  0.00297029]
  [-0.00024984 -0.0019097   0.00483129 -0.00610575]]

 [[-0.01301447  0.03369505  0.047205

In [3]:
class FMComponent(tf.keras.layers.Layer):
    def __init__(self,feat_dim,field_num,emb_dim,*args,**kwargs):
        super(FMComponent,self).__init__(*args,**kwargs)
        self.feat_dim=feat_dim
        self.field_num=field_num
        self.emb_dim=emb_dim

    def build(self, input_shape):
        self.w=tf.Variable(initial_value=tf.random.truncated_normal(shape=[self.feat_dim,]))

    def call(self, inputs, **kwargs):
        """

        :param inputs: (raw_input_batch,emb_vectors) ，其中raw_input_batch是feat_indices_batch,feat_value_batch 用于计算一阶term
            后者是emb_layer的输出
        :param kwargs:
        :return:
        """
        raw_input_batch,emb_vectors=inputs # emb_vectors: [batch_size, field_num, emb_dim]
        # feat_indices_batch: [batch_size, field_num]
        # feat_value_batch: [batch_size, field_num]
        feat_indices_batch,feat_value_batch=raw_input_batch

        # first order term 一阶做的事情实际上就是，假如一个样本有三个field上的取值为[1,1,1.3]，特征id分别是1，3，5，那么一阶结果就是w1*1+w3*1+w5*1.3
        # 使用feat_indices找到embedding_lookup快速找到field_num个权重然后做相乘
        # 例如，如果一个样本x，他在特征维度1、3、5上有取值，那么他的feat_indices=[1,3,5]。那只需要从self.w找到第1、3、5个数就可以了
        # 这样的计算方法更加快速
        weights=tf.nn.embedding_lookup(params=self.w,ids=tf.cast(feat_indices_batch,tf.int32)) # [batch_size, field_num]
        first_order_term = tf.multiply(feat_value_batch,weights) # [batch_size, field_num]
        first_order_term = tf.reduce_sum(first_order_term,axis=1,keepdims=True) # [batch_size, 1]

        # second order term
        # 下面这个是fm算法的优化算法 和平方减去平方和
        sum_square=tf.square(tf.reduce_sum(emb_vectors,axis=1)) # [batch_size, emb_dim]
        square_sum=tf.reduce_sum(tf.square(emb_vectors),axis=1) # [batch_size, emb_dim]

        second_order_term=1/2*tf.reduce_sum(tf.subtract(sum_square,square_sum),axis=1,keepdims=True) # [batch_size, 1]
        y_fm=first_order_term+second_order_term
        return y_fm

feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)


print("\nemb vectors")
emb_layer=EmbeddingLayer(feat_dim=10,field_num=3,emb_dim=4)
emb_vectors=emb_layer(input_batch)
print(emb_vectors)

fm_component=FMComponent(feat_dim=10,field_num=3,emb_dim=4)
fm_inputs=(input_batch,emb_vectors)
fm_outputs=fm_component(fm_inputs)
print("\nfm_outputs")
print(fm_outputs)



emb vectors
tf.Tensor(
[[[-3.38321105e-02 -2.09403988e-02 -6.26518577e-03 -3.71869430e-02]
  [-1.25457421e-02  1.19530037e-03  1.46252774e-02  3.13664190e-02]
  [-1.60620864e-02 -2.11294740e-02 -5.15757315e-03 -9.92386416e-03]]

 [[ 2.71561630e-02 -4.72365506e-02 -2.44090445e-02 -5.96357509e-03]
  [ 1.41158365e-02  4.28544320e-02  1.30845942e-02 -9.64999199e-05]
  [ 2.30111331e-02 -2.66944915e-02  3.33101191e-02  3.38966809e-02]]

 [[-1.25457421e-02  1.19530037e-03  1.46252774e-02  3.13664190e-02]
  [ 4.10801880e-02  3.52494381e-02 -4.03116457e-02 -2.21133232e-03]
  [ 2.57523600e-02 -2.98745017e-02  3.72782275e-02  3.79346609e-02]]

 [[ 1.41158365e-02  4.28544320e-02  1.30845942e-02 -9.64999199e-05]
  [ 2.71561630e-02 -4.72365506e-02 -2.44090445e-02 -5.96357509e-03]
  [-1.72213651e-03  1.64077210e-04  2.00759131e-03  4.30562440e-03]]

 [[ 3.20070274e-02 -3.71303447e-02  4.63322662e-02  4.71481346e-02]
  [ 2.71561630e-02 -4.72365506e-02 -2.44090445e-02 -5.96357509e-03]
  [ 1.55132459e-

In [4]:
class DeepComponent(tf.keras.layers.Layer):
    def __init__(self,deep_units_list,*args,**kwargs):
        super(DeepComponent,self).__init__(*args,**kwargs)
        self.deep_layers=list()
        for deep_units in deep_units_list:
            self.deep_layers.append(tf.keras.layers.Dense(units=deep_units,activation=tf.nn.relu))
        self.scoring_layer=tf.keras.layers.Dense(units=1,activation=None)

    def call(self, inputs, **kwargs):
        for deep_layer in self.deep_layers:
            inputs=deep_layer(inputs)
        outputs = self.scoring_layer(inputs)
        return outputs


In [5]:
class DeepFM(tf.keras.Model):
    def __init__(self,feat_dim,field_num,emb_dim,deep_units_list,*args,**kwargs):
        super(DeepFM,self).__init__(*args,**kwargs)

        self.emb_layer=EmbeddingLayer(feat_dim=feat_dim,field_num=field_num,emb_dim=emb_dim)
        self.fm_component=FMComponent(feat_dim=feat_dim,field_num=field_num,emb_dim=emb_dim)
        self.deep_component=DeepComponent(deep_units_list=deep_units_list)

    def call(self, inputs, training=None, mask=None):
        emb_vectors=self.emb_layer(inputs)

        fm_inputs=(inputs,emb_vectors)
        y_fm=self.fm_component(fm_inputs)

        deep_inputs=tf.reshape(emb_vectors,shape=[emb_vectors.shape[0],-1])
        y_deep=self.deep_component(deep_inputs)
        y=tf.nn.sigmoid(y_fm+y_deep)

        return y


feat_indices_arr=[np.random.choice(range(10),size=[1,3],replace=False) for _ in range(9)]
feat_indices_arr=np.concatenate(feat_indices_arr,axis=0).astype(np.float32)
feat_vals_arr=np.concatenate((np.ones(shape=[9,2]),
                              np.random.random(size=[9,1])),axis=1).astype(np.float32)

input_ds=tf.data.Dataset.from_tensor_slices((feat_indices_arr,feat_vals_arr))
batched_ds=input_ds.batch(5)
iterator=iter(batched_ds)
input_batch=next(iterator)

deep_fm_model=DeepFM(feat_dim=10,field_num=3,emb_dim=4,deep_units_list=[10,8])
deep_fm_model(input_batch)

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.33837795],
       [0.2670744 ],
       [0.4086375 ],
       [0.2040548 ],
       [0.27188796]], dtype=float32)>